```{contents}
```
## Session

In LangGraph, a **session** represents a **long-lived execution context** that groups together multiple graph invocations under a single logical interaction.
It is the core mechanism that enables **state persistence, memory, continuity, recovery, and multi-turn workflows**.

---

### **1. Intuition: Why Sessions Exist**

Without sessions, every graph invocation is **stateless**:

```
User Request → Graph → Output → Forget Everything
```

With sessions, LangGraph supports **continuity**:

```
Session Created
   ↓
Invocation 1 → State Update
   ↓
Invocation 2 → Uses Previous State
   ↓
Invocation 3 → Continues Workflow
```

Sessions make LangGraph behave like a **stateful application** rather than a single function call.

---

### **2. Formal Definition**

A **session** is:

> A persistent container that binds together **state, memory, execution history, checkpoints, and metadata** across multiple runs of a LangGraph.

---

### **3. Core Components of a Session**

| Component        | Purpose                          |
| ---------------- | -------------------------------- |
| Session ID       | Unique identifier for continuity |
| Thread ID        | Identifies conversation/workflow |
| State Store      | Holds current state              |
| Checkpoint Store | Saves intermediate states        |
| Execution Log    | Records transitions              |
| Memory           | Long-term storage                |
| Metadata         | User, timestamps, config         |

---

### **4. Lifecycle of a Session**

| Phase      | Description                    |
| ---------- | ------------------------------ |
| Create     | Session ID generated           |
| Initialize | Initial state stored           |
| Invoke     | Graph runs using session state |
| Persist    | State checkpoint saved         |
| Resume     | Next invocation loads state    |
| Terminate  | Session archived or deleted    |

---

### **5. How Sessions Work in Practice**

LangGraph sessions are controlled using the **`config`** argument.

```python
config = {
    "configurable": {
        "thread_id": "user-123"
    }
}
```

Each invocation using the same `thread_id` belongs to the **same session**.

```python
graph.invoke({"input": "Hello"}, config=config)
graph.invoke({"input": "Continue"}, config=config)
```

The second call automatically receives the updated state from the first.

---

### **6. State Persistence with Sessions**

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("sessions.db")

graph = builder.compile(checkpointer=checkpointer)
```

This enables:

* Crash recovery
* Session resumption
* Multi-turn memory
* Distributed execution

---

### **7. Example: Multi-Turn Session Workflow**

```python
class State(TypedDict):
    messages: list[str]

def chat_node(state):
    state["messages"].append("New message")
    return {"messages": state["messages"]}

builder = StateGraph(State)
builder.add_node("chat", chat_node)
builder.set_entry_point("chat")
builder.add_edge("chat", END)

graph = builder.compile(checkpointer=SqliteSaver("chat.db"))

config = {"configurable": {"thread_id": "alice"}}

graph.invoke({"messages": []}, config=config)
graph.invoke({}, config=config)
```

The second call sees the messages from the first call.

---

### **8. Session vs Stateless Execution**

| Feature        | Stateless | Session      |
| -------------- | --------- | ------------ |
| Memory         | ❌ None    | ✅ Persistent |
| Recovery       | ❌ No      | ✅ Yes        |
| Multi-turn     | ❌ No      | ✅ Yes        |
| Debugging      | ❌ Limited | ✅ Full trace |
| Production use | ❌ Rare    | ✅ Standard   |

---

### **9. Production Use Cases**

| Use Case            | Why Sessions          |
| ------------------- | --------------------- |
| Chat systems        | Maintain conversation |
| Agents              | Long-running autonomy |
| Workflow engines    | Multi-step pipelines  |
| Human-in-the-loop   | Pause & resume        |
| Compliance systems  | Full audit trail      |
| Distributed systems | Fault tolerance       |

---

### **10. Advanced Session Capabilities**

* Parallel sessions
* Session isolation
* Versioned state
* Session migration
* Time-travel debugging
* Cross-session memory

---

### **11. Mental Model**

Think of a LangGraph session as:

> **A database-backed execution container for intelligent workflows**

It is what turns LangGraph from a script into a **reliable production system**.

### Demonstration

In [12]:
from typing import TypedDict, List, Annotated
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import InMemorySaver

class ChatState(TypedDict):
    messages: List[str]

from typing import TypedDict, List

class ChatState(TypedDict):
    messages: List[str]

def chat_node(state: ChatState):
    state["messages"].append("Hello from the agent")
    return {"messages": state["messages"]}

from langgraph.graph import StateGraph, END

builder = StateGraph(ChatState)
builder.add_node("chat", chat_node)
builder.set_entry_point("chat")
builder.add_edge("chat", END)

from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": "user_alice"
    }
}

output1 = graph.invoke({"messages": []}, config=config)
print(output1)


{'messages': ['Hello from the agent']}


In [13]:
print(graph.invoke({}, config=config))
print(graph.invoke({}, config=config))

{'messages': ['Hello from the agent', 'Hello from the agent']}
{'messages': ['Hello from the agent', 'Hello from the agent', 'Hello from the agent']}
